# dataset Pre-process
Convert multiple labeled masks to a black/white mask of a single object

In [19]:
import numpy as np
import argparse
import cv2
import matplotlib.pyplot as plt
import os

## load images

In [ ]:
# Select data type
T_DATA=" ../NUbotsDataSet/train"
#T_DATA="../NUbotsDataSet/val"
imgLoadPath=T_DATA+"/imgs/"
imgMaskLoadPath=T_DATA+"/mask/"
imgSavePath=T_DATA+"/classMask/"

In [4]:
# resize
# 
from PIL import Image
import os.path
import glob

def convertjpg(jpgfile,outdir,width=640,height=512):
    img=Image.open(jpgfile)
    try:
        new_img=img.resize((width,height),Image.BILINEAR)   
        new_img.save(os.path.join(outdir,os.path.basename(jpgfile)))
    except Exception as e:
        print(e)
for jpgfile in glob.glob(imgLoadPath+"/*.jpg"): 
    convertjpg(jpgfile,T_DATA+"/resize/imgs/")
for jpgfile in glob.glob(imgMaskLoadPath+"/*.png"): 
    convertjpg(jpgfile,T_DATA+"/resize/mask/")

In [20]:
imgMaskLoadPath = T_DATA+"/resize/mask/"
imgSavePath = T_DATA+"/resize/classMask/"
imgLoadPath = T_DATA+"/resize/imgs/"
imgList=os.listdir(imgMaskLoadPath) #list of images mask
print("load from "+imgMaskLoadPath+", Total:",len(imgList))

Total: 1


In [21]:

# define color of class
goalColor = np.array([0,255,255]) # yellow
fieldColor = np.array([0,255,0]) #  green
markingColor = np.array([255,255,255]) # white
ballColor = np.array([0,0,255]) # red
#BGColor = np.array([0,0,0]) # black
# group
# className=["goal", "field", "marking", "ball", "BG"]
# classColor = [goalColor, fieldColor, markingColor, ballColor, BGColor]
className=["goal", "field", "marking", "ball"]
classColor = [goalColor, fieldColor, markingColor, ballColor]
cImg=0
cAllObj=0
# separate all images lables
for imgName in imgList: 
    image = cv2.imread(imgMaskLoadPath+imgName) # using openCV read image mask file
    classMasks=[] # Store mask of different classes
    cImg+=1
    cObj=0
    for item in zip(className,classColor):# find and separate each class 
        #classMasks.append(cv2.inRange(image, item[1], item[1]))
        if cv2.inRange(image, item[1], item[1]).any() > 0: 
            cObj+=1
            cAllObj+=1
            cv2.imwrite(imgSavePath+imgName.split(".")[0]+"_{}.png".format(item[0]), cv2.inRange(image, item[1], item[1]))
    print("%s : %d objects" %(imgName,cObj))
print("Total: %d images, %d objects" %(cImg,cAllObj))

0000005002_L.png : 4 objects
Total: 1 images, 4 objects


In [22]:
# # show
# # cv2.imshow("images", np.hstack([image, output]))
# # cv2.waitKey(0)

#mask = classMasks[0]

# plt.figure(figsize=(10,5)) 
# plt.subplot(2,3,1), plt.title('image')
# plt.imshow(image_merged), plt.axis('off')

# plt.subplot(2,3,2), plt.title('image_gray')
# plt.imshow(mask,cmap='gray'), plt.axis('off')

# plt.subplot(2,3,3), plt.title('image_merged')
# plt.imshow(image_merged), plt.axis('off')

# plt.subplot(2,3,4), plt.title('ball')
# plt.imshow(r,cmap='gray'), plt.axis('off')

# plt.subplot(2,3,5), plt.title('goal')
# plt.imshow(g,cmap='gray'), plt.axis('off')

# plt.subplot(2,3,6), plt.title('field')
# plt.imshow(b,cmap='gray'), plt.axis('off')
# plt.show()

# cv2.imwrite('yellow.png', mask)

# JSON

In [26]:
# dataset png to json

import datetime
import json
import os
import re
import fnmatch
from PIL import Image
import numpy as np
from pycococreatortools import pycococreatortools

ROOT_DIR = imgLoadPath
print(ROOT_DIR)
IMAGE_DIR = os.path.join(ROOT_DIR, "imgs")
ANNOTATION_DIR = os.path.join(ROOT_DIR, "classMask")

INFO = {
    "description": "NUbots_Dataset",
    "url": "null",
    "version": "0.1",
    "year": 2020,
    "contributor": "Bibby",
    "date_created": datetime.datetime.utcnow().isoformat(' ')
}

LICENSES = [
    {
        "id": 1,
        "name": "null",
        "url": "null"
    }
]
# not sure if need Background 
CATEGORIES = [
    {
        'id': 1,
        'name': 'goal',
        'supercategory': 'item',
    },
    {
        'id': 2,
        'name': 'field',
        'supercategory': 'item',
    },
    {
        'id': 3,
        'name': 'marking',
        'supercategory': 'item',
    },
    {
        'id': 4,
        'name': 'ball',
        'supercategory': 'item',
    },
]

def filter_for_jpeg(root, files):
    file_types = ['*.jpeg', '*.jpg']
    file_types = r'|'.join([fnmatch.translate(x) for x in file_types])
    files = [os.path.join(root, f) for f in files]
    files = [f for f in files if re.match(file_types, f)]
    
    return files

def filter_for_annotations(root, files, image_filename):
    file_types = ['*.png']
    file_types = r'|'.join([fnmatch.translate(x) for x in file_types])
    basename_no_extension = os.path.splitext(os.path.basename(image_filename))[0]
    file_name_prefix = basename_no_extension + '.*'
    files = [os.path.join(root, f) for f in files]
    files = [f for f in files if re.match(file_types, f)]
    files = [f for f in files if re.match(file_name_prefix, os.path.splitext(os.path.basename(f))[0])]

    return files

def main():

    coco_output = {
        "info": INFO,
        "licenses": LICENSES,
        "categories": CATEGORIES,
        "images": [],
        "annotations": []
    }

    image_id = 1
    segmentation_id = 1
    
    # filter for jpeg images
    for root, _, files in os.walk(IMAGE_DIR):
        image_files = filter_for_jpeg(root, files)

        # go through each image
        for image_filename in image_files:
            image = Image.open(image_filename)
            image_info = pycococreatortools.create_image_info(
                image_id, os.path.basename(image_filename), image.size)
            coco_output["images"].append(image_info)

            # filter for associated png annotations
            for root, _, files in os.walk(ANNOTATION_DIR):
                annotation_files = filter_for_annotations(root, files, image_filename)

                # go through each associated annotation
                for annotation_filename in annotation_files:
                    
                    print(annotation_filename)
                    class_id = [x['id'] for x in CATEGORIES if x['name'] in annotation_filename][0]

                    category_info = {'id': class_id, 'is_crowd': 'crowd' in image_filename}
                    binary_mask = np.asarray(Image.open(annotation_filename)
                        .convert('1')).astype(np.uint8)
                    
                    annotation_info = pycococreatortools.create_annotation_info(
                        segmentation_id, image_id, category_info, binary_mask,
                        image.size, tolerance=2)

                    if annotation_info is not None:
                        coco_output["annotations"].append(annotation_info)

                    segmentation_id = segmentation_id + 1

            image_id = image_id + 1

    with open('{}/NUbots_Dataset_A.json'.format(ROOT_DIR), 'w') as output_json_file:
        json.dump(coco_output, output_json_file)


if __name__ == "__main__":
    main()
    print("Done")

../NUbotDataSet/val
../NUbotDataSet/val\classMask\0000005002_L_ball.png
../NUbotDataSet/val\classMask\0000005002_L_field.png
../NUbotDataSet/val\classMask\0000005002_L_goal.png
../NUbotDataSet/val\classMask\0000005002_L_marking.png
Done


In [24]:
print(CATEGORIES)

[{'id': 1, 'name': 'goal', 'supercategory': 'item'}, {'id': 2, 'name': 'field', 'supercategory': 'item'}, {'id': 3, 'name': 'marking', 'supercategory': 'item'}, {'id': 4, 'name': 'ball', 'supercategory': 'item'}]
